In [1]:
import pandas as pd
import sys  
import os

directory = os.path.abspath('..')
sys.path.insert(0,directory)

from train import model_train, export_model_trained
from dbConnectionManager import Db_connection_manager
from predict import predict_model



In [2]:
manager_connection = Db_connection_manager()
manager_connection.open()

connection successful


In [3]:
# execute query and save data into var df
df = manager_connection.execute_query('''
select *
from fraudes;
''')

df = pd.DataFrame(df)
df[['fraudfound_p', 'repnumber', 'yearr', 'driverrating', 'deductible']] = df[['fraudfound_p', 'repnumber',
                                                                                  'yearr', 'driverrating', 'deductible']].astype(str)
df.head()

,monthh,weekofmonth,dayofweek,make,accidentarea,dayofweekclaimed,monthclaimed,weekofmonthclaimed,sex,maritalstatus,...,ageofvehicle,ageofpolicyholder,policereportfiled,witnesspresent,agenttype,numberofsuppliments,addresschange_claim,numberofcars,yearr,basepolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,...,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,...,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,...,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,...,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,...,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision


In [4]:
# Entrenando modelo
model = model_train(df)
model

Tamanio del X train: (12336, 32)
Tamanio del y train: (12336,)
Training model...


Pipeline(steps=[('pre',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('drop_columns', 'drop',
                                                  ['policynumber', 'repnumber',
                                                   'numberofsuppliments',
                                                   'addresschange_claim',
                                                   'policereportfiled',
                                                   'witnesspresent',
                                                   'agenttype']),
                                                 ('num',
                                                  Pipeline(steps=[('transfor_vars',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['age'], dtype='object')),
                                                 ('cat',...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=6, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=0, num_parallel_tree=1,
                               predictor='auto', random_state=0, reg_alpha=0,
                               reg_lambda=1, ...))])

In [5]:
# Exportando model entrenado
export_model_trained(model, f"{directory}/models")

exportando modelo...
model saved


In [12]:
model.predict_proba(df.head())

array([[9.9892563e-01, 1.0743570e-03],
       [8.6191159e-01, 1.3808839e-01],
       [9.8723066e-01, 1.2769342e-02],
       [9.9968851e-01, 3.1147612e-04],
       [9.9854642e-01, 1.4535533e-03]], dtype=float32)

In [ ]:
predict_model(f"{directory}/models", df.head())

/home/marcelamu9/.local/lib/python3.10/site-packages/sklearn/pipeline.py:764: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if est is None or est == "passthrough":


Pipeline(steps=[('predict_proba',
                 array([[9.9892563e-01, 1.0743570e-03],
       [8.6191159e-01, 1.3808839e-01],
       [9.8723066e-01, 1.2769342e-02],
       [9.9968851e-01, 3.1147612e-04],
       [9.9854642e-01, 1.4535533e-03]], dtype=float32))])